In [1]:
from datasets import load_dataset

d:\app\anaconda\envs\webui\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import random

In [12]:
a = [1, 2, 3, 4, 100]
# shuffle but not in place
b = random.sample(a, len(a))

In [13]:
b

[4, 1, 3, 100, 2]

In [ ]:
import torch
from torchmetrics.functional import stat_scores
from torchmetrics import Accuracy

# 假设 y_true 和 y_pred 是 PyTorch tensor 格式的标签
y_true = torch.tensor([[1, 0, 1], [0, 1, 1], [1, 1, 0]])
y_pred = torch.tensor([[1, 1, 0], [0, 1, 1], [1, 1, 0]])

# 使用 TorchMetrics 计算多标签F1分数
acc = Accuracy(num_classes=3, task="binary", num_labels=3, average="none")
f1 = acc(y_pred, y_true)
print(f"F1 Score: {f1}")

In [1]:
from datasets import load_dataset

ds = load_dataset("E:/pretrained_models/datasets/figer/figer/1.0.0")

d:\app\anaconda\envs\webui\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
ds.push_to_hub("DGME/figer", token="")

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.02it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/DGME/figer/commit/30d30988600b1aafe01208d48936680d1cce0b6a', commit_message='Upload dataset', commit_description='', oid='30d30988600b1aafe01208d48936680d1cce0b6a', pr_url=None, pr_revision=None, pr_num=None)

In [14]:
print(ds["train"][3]["mentions"])

{'start': [8, 13, 19], 'end': [11, 17, 23], 'labels': [['/train'], ['/organization', '/organization/company'], ['/rail/railway', '/organization', '/organization/company', '/rail', '/location']]}


In [11]:
ds["train"][:2]

{'tokens': [['It',
   'includes',
   'a',
   'cover',
   'of',
   '"',
   'Low',
   'Rider',
   '"',
   'by',
   'War',
   'and',
   'a',
   'reworking',
   'of',
   'Muddy',
   'Waters',
   "'",
   '"',
   'I',
   'Just',
   'Wanna',
   'Make',
   'Love',
   'To',
   'You',
   '"',
   'as',
   '"',
   'I',
   'Just',
   'Wanna',
   'Get',
   'High',
   'with',
   'You',
   '"',
   '.'],
  ['He',
   'was',
   'hospitalized',
   'for',
   'four',
   'days',
   'at',
   'the',
   'Mayo',
   'Clinic',
   'in',
   '1964',
   ',',
   'and',
   'for',
   'three',
   'weeks',
   'in',
   '1966',
   '.']],
 'senid': [0, 1],
 'mentions': [{'start': [15],
   'end': [17],
   'labels': [['/person/musician',
     '/person/actor',
     '/person/artist',
     '/person']]},
  {'start': [8],
   'end': [10],
   'labels': [['/organization',
     '/organization/company',
     '/building/hospital',
     '/building',
     '/location']]}],
 'fileid': ['train.jsonl', 'train.jsonl']}

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("E:/pretrained_models/Qwen/Qwen2___5-0___5B-Instruct")

In [3]:
tokenizer.apply_chat_template(
    [{"role": "user", "content": "I want to book a restaurant for 2 people."}],
    tokenize=False
)

'<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nI want to book a restaurant for 2 people.<|im_end|>\n'

In [23]:
from modules import register_template, AutoCLS
from concurrent.futures import ThreadPoolExecutor

def process(
    examples: dict, 
    tokenizer: AutoTokenizer,
    order: int, 
) -> dict:
    mentions, tokens = examples["mentions"], examples["tokens"]
    text, label, num_samples = [], [], len(mentions)
    
    ord = ["xx", "y"]
    
    def access_single(i: int):
        t, l = [], []
        mention, token = mentions[i], tokens[i] # there may be multiple mentions
        num_entity = len(mention["start"])
        for j in range(num_entity):
            start, end, labels = mention["start"][j], mention["end"][j], mention["labels"][j]
            sentence = token
            entity = sentence[start:end]
            t.append(register_template(
                sentence=" ".join(sentence), mention=" ".join(entity), cls_ord=ord
            ))
            l.append(labels)
        return t, l
   
    with ThreadPoolExecutor() as executor:
        res = executor.map(access_single, range(num_samples))
        for t, l in res: text.extend(t), label.extend(l)
        
    return {"text": text, "label": label}

In [24]:
x =process(
    ds["train"][:2], 
    tokenizer, 
    0
)

In [40]:
print(x["text"][0])

[Task]: Fine-grained entity classification
[sentence]: It includes a cover of " Low Rider " by War and a reworking of Muddy Waters ' " I Just Wanna Make Love To You " as " I Just Wanna Get High with You " .
[entity]: Muddy Waters
[entity types]: xx y
[Classification Result]: 
[Warning]: Just output nothing except entity types above, separate them by one space, there may be more than one answer


In [12]:
print(x["label"][1][0])

['/organization', '/organization/company', '/building/hospital', '/building', '/location']


In [17]:
ds["train"]

Dataset({
    features: ['tokens', 'senid', 'mentions', 'fileid'],
    num_rows: 1505765
})

In [21]:
a = [1, 2, 3]
a.extend([4, 5])

In [22]:
a

[1, 2, 3, 4, 5]

In [ ]:
from transformers import pipeline
pipe = pipeline("text-generation", "E:/pretrained_models/Qwen/Qwen2___5-0___5B-Instruct", 
    device_map="auto", torch_dtype="auto"
)


In [42]:
messages = [
    {"role": "system", "content": "You are a excellent linguist, you can finish the following task well! Also, you need to recognize some entity types are relative."},
    {"role": "user", "content": x["text"][0]},
]
print(pipe(messages, max_new_tokens=256)[0]['generated_text'][-1])

{'role': 'assistant', 'content': 'xx war'}


In [43]:
a = "123"

In [44]:
a.rfind('/')

-1

In [4]:
from datasets import load_from_disk

ds = load_from_disk("E:/nlp/toy/temp_exp/sft_data/figertest")

In [7]:
ds[0]

{'instruction': '[Task]: Fine-grained entity classification\n[sentence]: A handful of professors in the UW Department of Chemistry are being recognized by the American Association for the Advancement of Science ( AAAS ) for their efforts and contributions to the scientific community .\n[entity]: UW\n[entity types]: / person musician artist director author athlete actor architect doctor politician soldier engineer monarch coach religious_leader terrorist location city country cemetery province body_of_water county bridge language organization company sports_league sports_team educational_institution airline terrorist_organization fraternity_sorority people ethnicity written_work software product computer weapon airplane ship spacecraft car instrument mobile_phone engine_device camera government government political_party park government_agency art film broadcast_program game geography island mountain glacier music military train rail railway building hospital airport sports_facility res